<a href="https://colab.research.google.com/github/BrithneyHernandez25/BrithneyHernandez.github.io/blob/main/Prediccion_de_sentimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Autenticación e importación de bibliotecas necesarias
from google.colab import auth
auth.authenticate_user()
import gspread
import pandas as pd
from google.auth import default
from datetime import datetime

# Autorización de credenciales
creds, _ = default()
gc = gspread.authorize(creds)

# Definir la URL del archivo de Google Sheets
url = "https://docs.google.com/spreadsheets/d/1noPa-dmGgXcBcoJx-mEsEZRY0ub7H0kdeMkqx2oaNDo/edit#gid=0"

# Obtener el ID del documento
doc_id = url.split("/")[5]
print("Documento ID:", doc_id)

# Abrir el documento y seleccionar la hoja de trabajo
doc = gc.open_by_key(doc_id)
ws = doc.worksheet("Sentimiento")

# Obtener todos los valores de la hoja
values = ws.get_all_values()

# Crear un DataFrame a partir de los valores, usando la segunda fila como encabezados
df = pd.DataFrame(values[2:], columns=values[1])
df.dropna(inplace=True)

# Verificar si las columnas 'Headline' y 'Sentiment' existen en el DataFrame
columnas_requeridas = ["Headline", "Sentiment"]
for col in columnas_requeridas:
    if col not in df.columns:
        raise ValueError(f"La columna requerida '{col}' no está presente en el DataFrame")

# Filtrar el DataFrame para mantener solo las columnas necesarias
df = df[columnas_requeridas]

# Mostrar los primeros datos del DataFrame para verificación
print(df.head())

# Verificar los valores únicos en la columna 'Sentiment'
print(df['Sentiment'].unique())

# Eliminar filas con sentimientos vacíos
df = df[df['Sentiment'].isin(['Positive', 'Negative', 'Mixto'])]

# Codificar las etiquetas de sentimiento
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
df['Sentiment'] = label_encoder.fit_transform(df['Sentiment'])

# Verificar la codificación de los sentimientos
print(df['Sentiment'].unique())

# Codificar a categorías
sentimientos_categoricos = to_categorical(df['Sentiment'], num_classes=3)

# Convertir los encabezados a una lista de cadenas
encabezados = df['Headline'].tolist()

# Tokenizar y secuenciar los encabezados
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(encabezados)
sequences = tokenizer.texts_to_sequences(encabezados)
padded_sequences = pad_sequences(sequences, maxlen=50)

# Dividir los datos en conjuntos de entrenamiento y prueba (75% entrenamiento, 25% prueba)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, sentimientos_categoricos, test_size=0.25, random_state=42)

# Definir la arquitectura del modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D

modelo = Sequential()
modelo.add(Embedding(input_dim=5000, output_dim=128, input_length=50))
modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
modelo.add(Dense(3, activation='softmax'))

# Compilar el modelo
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Mostrar resumen del modelo
modelo.summary()

# Entrenar el modelo
from tensorflow.keras.callbacks import EarlyStopping

epocas = 5
tamano_lote = 64

historia = modelo.fit(X_train, y_train, epochs=epocas, batch_size=tamano_lote, validation_data=(X_test, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

# Evaluar el modelo
perdida, precision = modelo.evaluate(X_test, y_test, verbose=2)
print(f'Precisión en el conjunto de prueba: {precision:.2f}')

# Función para predecir el sentimiento de nuevos encabezados
def predecir_sentimiento(nuevo_encabezado):
    secuencia = tokenizer.texts_to_sequences([nuevo_encabezado])
    secuencia_padded = pad_sequences(secuencia, maxlen=50)
    prediccion = modelo.predict(secuencia_padded)
    clase_predicha = prediccion.argmax(axis=-1)[0]
    sentimiento = label_encoder.inverse_transform([clase_predicha])[0]
    return sentimiento




Documento ID: 1noPa-dmGgXcBcoJx-mEsEZRY0ub7H0kdeMkqx2oaNDo
                                            Headline Sentiment
0  These are the streaming releases for January 2023     Mixto
1              Top Amazon Prime Video series in 2022     Mixto
2                       Series that return this 2022     Mixto
3  Where to watch Barack Obama's favorite movies ...     Mixto
4                                    Travel to a set     Mixto
['Mixto' 'Positive' 'Negative' '']
[0 2 1]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 128)           640000    
                                                                 
 spatial_dropout1d (Spatial  (None, 50, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 10